In [66]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import itertools
from collections import defaultdict

In [67]:
%matplotlib inline
plt.rcParams.update({'font.size': 18})
sns.set_style('whitegrid')

In [68]:
df = pd.read_csv('data/declarations_clean.csv')
df.head()

,document_type,party,person_id,person_name,year,office_id,office_name,office_post,office_type_id,office_type_name,office_region_id,office_region_name,income_personal,income_relatives
0,антикоррупционная декларация,кпрф,8,зюганов геннадий андреевич,2009,14,государственная дума,член комитета государственной думы,7,федеральный без региональной структуры,NaN,NaN,2075542.33,102279.19
1,антикоррупционная декларация,кпрф,8,зюганов геннадий андреевич,2010,14,государственная дума,член комитетa государственной думы,7,федеральный без региональной структуры,NaN,NaN,2127686.49,168601.94
2,антикоррупционная декларация,кпрф,8,зюганов геннадий андреевич,2011,14,государственная дума,NaN,7,федеральный без региональной структуры,NaN,NaN,2177566.64,137656.24
3,антикоррупционная декларация,кпрф,8,зюганов геннадий андреевич,2012,14,государственная дума,член комитета гд по науке и наукоемким техноло...,7,федеральный без региональной структуры,NaN,NaN,2458749.04,194266.38
4,антикоррупционная декларация,кпрф,8,зюганов геннадий андреевич,2013,14,государственная дума,член комитета государственной думы,7,федеральный без региональной структуры,NaN,NaN,3194615.67,198267.22


In [69]:
grouped_view = df.groupby(['person_id', 'person_name', 'year']).first()

In [70]:
common_offices = df.groupby(['year', 'office_id'])['person_id'].apply(list).reset_index()
common_offices.head()

,year,office_id,person_id
0,2009,1,"[9133, 9161]"
1,2009,3,"[679, 680, 681, 682, 7298, 7299, 7300, 7301, 7..."
2,2009,4,[5350]
3,2009,5,"[80, 299]"
4,2009,7,"[1882, 19701]"


In [71]:
# common_offices.to_csv('data/common_offices.csv', index=False)

In [72]:
worked_together_dict = defaultdict(int)    
for person_id_list in common_offices.person_id:
    person_id_list = [int(x) for x in person_id_list]
    for pair in itertools.combinations(person_id_list, 2):
        worked_together_dict[pair] += 1
    

len(worked_together_dict)

4437052

In [73]:
# assert worked_together_dict[(80, 299)] == 1

In [74]:
# More or equal to the amount of persons
# Less than the number of all possible pairs
unique_persons_count = df['person_id'].unique().shape[0]
print('Unique people:', unique_persons_count)
assert unique_persons_count <= len(worked_together_dict)
assert len(worked_together_dict) <= unique_persons_count*(unique_persons_count-1)/2 

Unique people: 51672


In [75]:
with open('data/graph.csv', 'w') as graph_file:
    graph_file.write('person1,person2,times_worked_together\n')
    for pair in sorted(worked_together_dict):
        graph_file.write('{},{},{}\n'.format(pair[0], pair[1], worked_together_dict[pair]))

In [76]:
import networkx as nx
df_graph = pd.read_csv('data/graph.csv', header=1)
graph = nx.Graph()
for v1, v2, weight in df_graph.values:
    graph.add_edge(v1, v2, weight=weight)
    
graph_nodes = graph.order() 
print(graph_nodes, 'graph nodes')
print(graph.size(), 'graph edges')

51143 graph nodes
4437051 graph edges


In [78]:
assert graph_nodes <= unique_persons_count